In [2]:
import numpy as np
import os

In [3]:
path_labels = r"D:\dataincendios\smokenet-projectx\cvat_labels"
path = r"C:\2024\smoke-detection-net\data\full\frames"

In [4]:

# get all files in the directory
files = os.listdir(path)
# group by name before _frame_

# Crear un diccionario para agrupar los archivos
grouped_files = {}

for file in files:
    # Obtener la parte antes de '_frame_'
    key = file.split('_frame_')[0]
    
    # Agregar el archivo a la lista correspondiente en el diccionario
    if key not in grouped_files:
        grouped_files[key] = []
    grouped_files[key].append(file)



In [5]:
len(grouped_files.keys())

643

In [8]:
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt

In [42]:
# Ruta donde se encuentran los archivos XML y frames
path_labels = r"D:\dataincendios\smokenet-projectx\cvat_labels"
path_frames = r"C:\2024\smoke-detection-net\data\full\frames"
output_directory = r"D:\dataincendios/smoke50"
# create output directory
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [43]:
# Iterar sobre cada grupo en el diccionario de archivos agrupados
for key, files in grouped_files.items():
    # Crear carpeta para los frames correspondientes
    # frames_folder = os.path.join(output_directory, key)
    frames_folder = os.path.join(output_directory, key.split('_')[0])
    # frame_folder son los nombres antes de _ mas 5 palabras desapues de _
    # frames_folder = os.path.join(output_directory, key.split('_')[0] + '_' + ''.join(key.split('_')[1].split(" ")[1:10]))
    # si no existe la carpeta la crea
    counter = 1
    original_folder = frames_folder
    while os.path.exists(frames_folder) and counter < 20:
        print(f"La carpeta {frames_folder} ya existe")
        frames_folder = f"{original_folder}_{counter}"
        counter += 1
    if counter == 20:
        print(f"Error al crear carpeta para el grupo {key}")
        continue
    os.makedirs(frames_folder)


    
    # Copiar cada frame del grupo a la carpeta correspondiente
    for file in files:
        frame_path = os.path.join(path_frames, file)
        if os.path.exists(frame_path):
            shutil.copy(frame_path, frames_folder)
        else:
            print(f"Frame no encontrado: {frame_path}")

    # Crear carpeta labels dentro de la carpeta del grupo
    # cambiar el nombre de frame folder por lo que va antes de _
    
    labels_folder = os.path.join(frames_folder, 'labels')
    if not os.path.exists(labels_folder):
        os.makedirs(labels_folder)
    
    # Leer el archivo XML correspondiente a las anotaciones
    example = os.path.join(path_labels, key + '.xml')

    try:
        with open(example, 'r') as file:
            data = file.read()

        # Parsear el archivo XML
        root = ET.fromstring(data)

        # Obtener las dimensiones de la imagen del XML
        img_width, img_height = None, None
        for original_size in root.iter('original_size'):
            img_width = int(original_size.find('width').text)
            img_height = int(original_size.find('height').text)
            break
        # tamaño de la imagen 

        if img_width is None or img_height is None:
            print(f"Dimensiones de imagen no encontradas en el archivo XML: {example}")
            continue

        # Crear un diccionario para almacenar los cuadros (box) de cada frame
        boxes_by_frame = {}

        for box in root.iter('box'):
            frame = box.get('frame')
            xtl = float(box.get('xtl'))
            ytl = float(box.get('ytl'))
            xbr = float(box.get('xbr'))
            ybr = float(box.get('ybr'))
            box_coordinates = (xtl, ytl, xbr, ybr)

            if frame not in boxes_by_frame:
                boxes_by_frame[frame] = []
            boxes_by_frame[frame].append(box_coordinates)

        # Iterar sobre los archivos en el grupo para generar los archivos de anotaciones
        for file in files:
            # Extraer el número de frame del nombre del archivo
            frame_number = file.split('_frame_')[1].split('.jpg')[0]

            if frame_number in boxes_by_frame:
                # Crear un archivo de texto para cada frame en la carpeta labels
                txt_filename = os.path.join(labels_folder, f"{file.split('.jpg')[0]}.txt")
                with open(txt_filename, 'w') as txt_file:
                    for box in boxes_by_frame[frame_number]:
                        # Calcular los parámetros en formato YOLO
                        xtl, ytl, xbr, ybr = box
                        width = xbr - xtl
                        height = ybr - ytl
                        x_center = xtl + width / 2
                        y_center = ytl + height / 2
                        
                        # Normalizar valores usando dimensiones de la imagen
                        x_center /= img_width
                        y_center /= img_height
                        width /= img_width
                        height /= img_height

                        # Escribir la clase (0 para smoke en este caso) y las coordenadas
                        txt_file.write(f"0 {x_center} {y_center} {width} {height}\n")



    except FileNotFoundError:
        print(f"Archivo XML no encontrado: {example}")

La carpeta D:\dataincendios/smoke50\20130711 ya existe
La carpeta D:\dataincendios/smoke50\20130711 ya existe
La carpeta D:\dataincendios/smoke50\20130711_1 ya existe
La carpeta D:\dataincendios/smoke50\20140916 ya existe
La carpeta D:\dataincendios/smoke50\20140916 ya existe
La carpeta D:\dataincendios/smoke50\20140916_1 ya existe
La carpeta D:\dataincendios/smoke50\20140916 ya existe
La carpeta D:\dataincendios/smoke50\20140916_1 ya existe
La carpeta D:\dataincendios/smoke50\20140916_2 ya existe
La carpeta D:\dataincendios/smoke50\20140916 ya existe
La carpeta D:\dataincendios/smoke50\20140916_1 ya existe
La carpeta D:\dataincendios/smoke50\20140916_2 ya existe
La carpeta D:\dataincendios/smoke50\20140916_3 ya existe
La carpeta D:\dataincendios/smoke50\20141003 ya existe
La carpeta D:\dataincendios/smoke50\20150131 ya existe
La carpeta D:\dataincendios/smoke50\20150131 ya existe
La carpeta D:\dataincendios/smoke50\20150131_1 ya existe
La carpeta D:\dataincendios/smoke50\20150201 ya e

In [ ]:
# 20150815_Cold Springs Incident fire, Day 2 seen from Fairview Peak at 6 PM, August 15th, 2015.mp4_frame_0
# 20150815_Cold Springs Incident fire, Day 2 seen from Fairview Peak at 6 PM, August 15th, 2015.mp4_frame_0


In [45]:
# ahora repartir los archivos en train, val y test
# 80% train, 10% val, 10% test
# por cada video 5 frames 4 de incendios y 1 sin incendio

# Crear un diccionario para agrupar los archivos
path_videos = r"D:\dataincendios\smoke50"
videos = os.listdir(path_videos)
# SORT VIDEOS
videos.sort()

n_train = int(len(videos) * 0.8)
n_val = int(len(videos) * 0.1)
n_test = int(len(videos) * 0.1)

# Crear listas para cada conjunto
train_videos = videos[:n_train]
val_videos = videos[n_train:n_train + n_val]
test_videos = videos[n_train + n_val:]

# hacer una copia de los videos train val y test

# Crear directorios para train, val y test
output_directory = r"D:\dataincendios\smoke50_split"
train_dir = os.path.join(output_directory, 'train')
val_dir = os.path.join(output_directory, 'val')
test_dir = os.path.join(output_directory, 'test')

# Crear directorios
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)

# Copiar los videos a los directorios correspondientes
for video in train_videos:
    video_path = os.path.join(path_videos, video)
    shutil.copytree(video_path, os.path.join(train_dir, video))

for video in val_videos:
    video_path = os.path.join(path_videos, video)
    shutil.copytree(video_path, os.path.join(val_dir, video))

for video in test_videos:
    video_path = os.path.join(path_videos, video)
    shutil.copytree(video_path, os.path.join(test_dir, video))


In [48]:
path_videos = r"D:\dataincendios\smoke50_split_videos"
path_images = r"D:\dataincendios\smoke50_split_images"

# videos
videos = os.listdir(path_videos)

# train
videos_train = os.listdir(os.path.join(path_videos, 'train'))
# val
videos_val = os.listdir(os.path.join(path_videos, 'val'))
# test
videos_test = os.listdir(os.path.join(path_videos, 'test'))

# Crear directorios para train, val y test
os.makedirs(os.path.join(path_images, 'train'), exist_ok=True)
os.makedirs(os.path.join(path_images, 'val'), exist_ok=True)
os.makedirs(os.path.join(path_images, 'test'), exist_ok=True)

# seleccionar 5 frames de cada video, 4 con incendios y 1 sin incendio, y copiarlos a la carpeta correspondiente
# donde los 4 frames con incendios se toma el primer frame con deteccion el ultimo y el resto al azar
# y el frame sin incendio se toma al azar
output_base = path_images
images_dir = os.path.join(output_base, 'images', 'train')
labels_dir = os.path.join(output_base, 'labels', 'train')

# Crear las carpetas para la estructura YOLO si no existen
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
# train
for video in videos_train:
    video_path = os.path.join(path_videos, 'train', video)
    labels_path = os.path.join(video_path, 'labels')

    # Obtener frames y labels
    frames = os.listdir(video_path)
    labels = [label for label in os.listdir(labels_path) if label.endswith('.txt')]

    # Verificar si hay labels
    if labels:
        # Ordenar labels
        labels.sort()

        # Seleccionar el primer y último frame con incendio
        first_frame = labels[0].replace('.txt', '.jpg')
        last_frame = labels[-1].replace('.txt', '.jpg')

        # Seleccionar 2 frames al azar entre los restantes con incendio (si hay más de 2)
        if len(labels)-2 > 2:
            random_frames = np.random.choice(labels[1:-1], 2)
            random_frames = [frame.replace('.txt', '.jpg') for frame in random_frames]
        else:
            random_frames = []

        # Seleccionar 1 frame sin incendio
        label_frames = set([label.replace('.txt', '.jpg') for label in labels])
        non_fire_frames = [frame for frame in frames if frame.endswith('.jpg') and frame not in label_frames]

        if non_fire_frames:
            non_fire_frame = np.random.choice(non_fire_frames, 1)[0]
        else:
            non_fire_frame = None

        # Resultado final: frames seleccionados
        selected_frames = [first_frame, last_frame] + random_frames
        if non_fire_frame:
            selected_frames.append(non_fire_frame)

        # Copiar los frames seleccionados y sus etiquetas correspondientes a la estructura YOLO
        for frame in selected_frames:
            # Copiar la imagen
            src_frame_path = os.path.join(video_path, frame)
            dst_frame_path = os.path.join(images_dir, frame)
            shutil.copy(src_frame_path, dst_frame_path)

            # Copiar la etiqueta correspondiente si existe
            label_filename = frame.replace('.jpg', '.txt')
            src_label_path = os.path.join(labels_path, label_filename)
            dst_label_path = os.path.join(labels_dir, label_filename)

            if os.path.exists(src_label_path):
                shutil.copy(src_label_path, dst_label_path)
            else:
                # Si el frame es uno sin incendio, se genera un archivo de anotación vacío
                with open(dst_label_path, 'w') as label_file:
                    pass

    else:
        # Caso cuando no hay incendios en los frames
        print(f"No hay frames con incendio para el video {video}")
        continue

print("Copias completadas con estructura YOLO.")


    



No hay frames con incendio para el video 20150131
No hay frames con incendio para el video 20150131_1
No hay frames con incendio para el video 20150201_1
No hay frames con incendio para el video 20150617
No hay frames con incendio para el video 20150617_1
No hay frames con incendio para el video 20150704
No hay frames con incendio para el video 20150918
No hay frames con incendio para el video 20160608
No hay frames con incendio para el video 20160608_1
No hay frames con incendio para el video 20160628_4
No hay frames con incendio para el video 20160630
No hay frames con incendio para el video 20160710
No hay frames con incendio para el video 20160717
No hay frames con incendio para el video 20160717_1
No hay frames con incendio para el video 20160719_1
No hay frames con incendio para el video 20160725
No hay frames con incendio para el video 20160812
No hay frames con incendio para el video 20170619_1
No hay frames con incendio para el video 20170623_3


ValueError: Cannot take a larger sample than population when 'replace=False'